In [17]:
# Model with Model_Params

In [18]:
import datetime as dt
import pandas as pd

from de.processor.source.dart import DART_FUNDAMENTAL_PROCESSOR
from de.processor.source.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR
from de.loader.api_loader.pykrx import get_pykrx_loader

from de.processor.factor.processor import (
    FUNDAMENTAL_FACTOR_PROCESSOR,
    MOVING_AVERAGE_FACTOR_PROCESSOR,
    LINEAR_COEF_FACTOR_PROCESSOR,
    TRADER_FACTOR_PROCESSOR,
)

from de.model_parameter import MODEL_PARAMTER_PROCESSOR

from de.processor.factor.analyser import FACTOR_ANALYSER
from de.processor.factor.filter import FACTOR_FILTER

In [19]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
        "FILTER": {},
    },
    "MOVING_AVERAGE": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        },
        "FILTER": {},
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        },
        "FILTER": {},
    },
    "TRADER": {
        "PROCESSOR": {
            "start": (dt.date.today() - dt.timedelta(days=9)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FILTER": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
    "FACTOR_ANALYSIS": {
        "profit_recent_n": 5,
    },
}
recen_n = CFG["FACTOR_ANALYSIS"]["profit_recent_n"]
analysis_date = (dt.date.today() - dt.timedelta(days=recen_n)).strftime("%Y-%m-%d")
CFG["analysis_date"] = analysis_date

In [20]:
# LOAD
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [21]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [22]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

# split ohlcv
future_ohlcv_df = ohlcv_df[CFG["analysis_date"] < ohlcv_df["Date"]].copy()
past_ohlcv_df = ohlcv_df[ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [23]:
model_paramter_processor = MODEL_PARAMTER_PROCESSOR(CFG["FACTOR_ANALYSIS"])

In [24]:
# Fundamental

## Parameter
### 1. Get Factor_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    past_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

### 2. Get Parameter
fundamental_factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)
fundamental_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, fundamental_factor_analyser)
fundamental_param_df = pd.DataFrame().from_dict(fundamental_params_dict, orient="index", columns=["Value", "Variance"])

### 3. Set Paramter
fundamental_params = fundamental_param_df.nlargest(6, "Value").nsmallest(3, "Variance").index

f_stockcodes_list = list()
for fundamental_param in fundamental_params:
    CFG["FUNDAMENTAL"]["FILTER"]["quantile_conditions"] = fundamental_param

    ## Model
    ### 1. Get Factor_df
    fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
        ohlcv_df, dart_fundamental_processor, fdr_info_processor
    )
    fundamental_factor_df = fundamental_factor_processor.get_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

    ### 2. Get Filtered_stockcodes
    factor_filter = FACTOR_FILTER(fundamental_factor_df)
    f_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FILTER"])
    f_stockcodes_list.append(f_stockcodes)

f_stockcodes = set(sum([list(i) for i in f_stockcodes_list], []))

In [25]:
# Technical
# Moving Average

## Paramter
### 1. Get Factor_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_factor_df(
    CFG["MOVING_AVERAGE"]["PROCESSOR"]
)

### 2. Get Parameter
moving_average_factor_analyser = FACTOR_ANALYSER(moving_average_factors_df)
moving_average_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, moving_average_factor_analyser)
moving_average_param_df = pd.DataFrame().from_dict(
    moving_average_params_dict, orient="index", columns=["Value", "Variance"]
)

### 3. Set Paramter
moving_average_params = moving_average_param_df.nlargest(6, "Value").nsmallest(3, "Variance").index

ma_stockcodes_list = list()
for moving_average_param in moving_average_params:
    CFG["MOVING_AVERAGE"]["FILTER"]["quantile_conditions"] = moving_average_param

    ## Model
    ### 1. Get Factor_df
    moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
    moving_average_factor_df = moving_average_factor_processor.get_factor_df(
        CFG["MOVING_AVERAGE"]["PROCESSOR"]
    )

    ### 2. Get Filtered_stockcodes
    factor_filter = FACTOR_FILTER(moving_average_factor_df)
    ma_stockcodes = factor_filter.get_filtered_stockcodes(CFG["MOVING_AVERAGE"]["FILTER"])
    ma_stockcodes_list.append(ma_stockcodes)

ma_stockcodes = set(sum([list(i) for i in ma_stockcodes_list], []))

In [26]:
# Technical
# Linear Coef

## Paramter
### 1. Get Factor_df
linear_coef_factor_processor = LINEAR_COEF_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
linear_coef_factors_df = linear_coef_factor_processor.get_factor_df(
    CFG["LINEAR_COEF"]["PROCESSOR"]
)

### 2. Get Parameter
linear_coef_factor_analyser = FACTOR_ANALYSER(linear_coef_factors_df)
linear_coef_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, linear_coef_factor_analyser)
linear_coef_param_df = pd.DataFrame().from_dict(
    linear_coef_params_dict, orient="index", columns=["Value", "Variance"]
)

### 3. Set Paramter
linear_coef_params = linear_coef_param_df.nlargest(6, "Value").nsmallest(3, "Variance").index

lc_stockcodes_list = list()
for linear_coef_param in linear_coef_params:
    CFG["LINEAR_COEF"]["FILTER"]["quantile_conditions"] = linear_coef_param

    ## Model
    ### 1. Get Factor_df
    linear_coef_factor_processor = LINEAR_COEF_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
    linear_coef_factor_df = linear_coef_factor_processor.get_factor_df(
        CFG["LINEAR_COEF"]["PROCESSOR"]
    )

    ### 2. Get Filtered_stockcodes
    factor_filter = FACTOR_FILTER(linear_coef_factor_df)
    lc_stockcodes = factor_filter.get_filtered_stockcodes(CFG["LINEAR_COEF"]["FILTER"])
    lc_stockcodes_list.append(lc_stockcodes)

lc_stockcodes = set(sum([list(i) for i in lc_stockcodes_list], []))

In [27]:
f_ma_stockcodes = f_stockcodes & ma_stockcodes
f_lc_stockcodes = f_stockcodes & lc_stockcodes

In [30]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)

## Fundamental - Linear Coef - Trader
trader_factor_df = trader_factor_processor.get_factor_df(f_lc_stockcodes, CFG["TRADER"]["PROCESSOR"])
factor_filter = FACTOR_FILTER(trader_factor_df)
f_lc_t_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], f_lc_stockcodes)

## Fundamental - Moving Avergae - Trader
trader_factor_df = trader_factor_processor.get_factor_df(f_ma_stockcodes, CFG["TRADER"]["PROCESSOR"])
factor_filter = FACTOR_FILTER(trader_factor_df)
f_ma_t_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], f_ma_stockcodes)

In [31]:
f_lc_t_stockcodes

{'023960', '298040'}

In [32]:
f_ma_t_stockcodes

{'047050', '063170', '066970', '079810'}